In [16]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
# for neural net
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report

In [2]:
data = pd.read_csv("data/combined_expression.csv")

In [3]:
data['classification'].replace({1: 0, 2: 1}, inplace=True)
data.head()

,CELL_LINE_NAME,classification,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,COL15A1,C6orf10,TMEM225,NOTCH4,PBX2,AGER,RNF5,AGPAT1,DFNB59,PRRT1
0,1240121,0,6.419526,3.182094,9.320548,3.759654,3.802619,3.215753,4.698729,7.873672,...,3.245454,2.953508,3.543429,3.352022,4.672310,3.641128,3.135310,3.737072,3.450927,3.168800
1,1240122,1,7.646494,2.626819,10.153853,3.564755,3.942749,3.290760,3.551675,8.252413,...,2.786709,3.077382,3.728232,3.208882,4.586840,3.395654,3.586800,3.519128,3.115323,3.051645
2,1240123,0,8.319417,3.111183,9.643558,4.757258,3.919757,3.602185,3.329644,9.076950,...,3.459089,3.085394,3.462811,3.339030,4.614897,3.395845,3.419193,3.971646,3.729310,3.320022
3,1240124,0,9.006994,3.028173,9.686700,4.280504,3.147646,3.188881,3.293807,8.678790,...,2.835403,2.960303,3.415083,3.290171,4.770123,3.400821,3.383734,3.798107,2.822404,3.297547
4,1240127,0,7.985676,2.694729,10.676134,4.159685,3.804637,3.481942,3.111261,7.555407,...,2.896523,2.849899,3.480114,3.226128,5.832710,3.612179,3.347095,4.457963,5.198524,4.553586


In [4]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [5]:
# retrieving proper columns
X = data.loc[:, selected_genes].values
y = data['classification'].values
# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)
# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# Gridsearch for Input and Output Layer (One Hidden Layer)

## Optimizing Epochs and Batches

In [6]:
def create_model(optimizer='adam'):
    model = Sequential()
    # adding layers
    model.add(Dense(len(selected_genes), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
model = KerasClassifier(build_fn=create_model)
epochs = [10, 25, 50, 100]
batches = [16, 32, 64, 128]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(epochs=epochs, batch_size=batches, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

/Users/eddieguo/anaconda3/envs/fixed_comp/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/100
513/513 [==============================] - 1s 1ms/step - loss: 0.7243 - accuracy: 0.5673
Epoch 2/100
513/513 [==============================] - 0s 77us/step - loss: 0.6347 - accuracy: 0.6608
Epoch 3/100
513/513 [==============================] - 0s 80us/step - loss: 0.5768 - accuracy: 0.6862
Epoch 4/100
513/513 [==============================] - 0s 72us/step - loss: 0.5867 - accuracy: 0.7388
Epoch 5/100
513/513 [==============================] - 0s 83us/step - loss: 0.5742 - accuracy: 0.6823
Epoch 6/100
513/513 [==============================] - 0s 77us/step - loss: 0.5606 - accuracy: 0.7505
Epoch 7/100
513/513 [==============================] - 0s 74us/step - loss: 0.5478 - accuracy: 0.7427
Epoch 8/100
513/513 [==============================] - 0s 77us/step - loss: 0.5570 - accuracy: 0.6979
Epoch 9/100
513/513 [==============================] - 0s 83us/step - loss: 0.5374 - accuracy: 0.7290
Epoch 10/100
513/513 [==============================] - 0s 84us/step - loss: 0.5663

513/513 [==============================] - 0s 85us/step - loss: 0.4481 - accuracy: 0.8051
Epoch 81/100
513/513 [==============================] - 0s 85us/step - loss: 0.4555 - accuracy: 0.7778
Epoch 82/100
513/513 [==============================] - 0s 83us/step - loss: 0.4487 - accuracy: 0.8051
Epoch 83/100
513/513 [==============================] - 0s 81us/step - loss: 0.4658 - accuracy: 0.8090
Epoch 84/100
513/513 [==============================] - 0s 85us/step - loss: 0.4588 - accuracy: 0.7817
Epoch 85/100
513/513 [==============================] - 0s 82us/step - loss: 0.4585 - accuracy: 0.7739
Epoch 86/100
513/513 [==============================] - 0s 79us/step - loss: 0.4891 - accuracy: 0.7719
Epoch 87/100
513/513 [==============================] - 0s 75us/step - loss: 0.4643 - accuracy: 0.7973
Epoch 88/100
513/513 [==============================] - 0s 76us/step - loss: 0.4408 - accuracy: 0.8031
Epoch 89/100
513/513 [==============================] - 0s 78us/step - loss: 0.4400 - 

In [10]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.752437 using {'batch_size': 64, 'epochs': 100, 'optimizer': 'Adamax'}


In [11]:
grid_result.cv_results_

{'mean_fit_time': array([ 5.2021636 ,  7.46684613,  5.95582743,  8.64542003,  8.56029124,
         6.56816416,  9.74208169,  8.42310057, 11.57348404, 11.12729182,
        17.79938416, 16.9462141 , 12.46522093, 17.255901  , 16.71501217,
        21.63394456, 20.38423309, 27.95739164, 26.08763938, 22.92758164,
        29.82075305, 34.97338004, 45.07815275, 43.23775382, 55.66169615,
        51.89609666, 43.23025951, 42.82431064,  5.67866254,  8.77549472,
         8.93743591, 10.85489421,  9.31100316,  5.63129916,  7.62295227,
         5.67463179,  7.66104136,  6.84610858,  9.48351941,  8.96711965,
         7.87698112, 11.1190383 ,  9.46409097, 12.01111097, 11.48419876,
        15.0754046 , 15.10124159, 12.66982522, 17.91575823, 18.53054519,
        21.84955144, 22.34677782, 27.95172658, 27.0053566 , 23.72991576,
        22.83535175,  4.89873881,  6.90173645,  6.41060648,  7.83425164,
         6.036378  ,  4.8538404 ,  7.13061953,  4.75712166,  5.95515823,
         5.34932022,  7.65910578, 

# Testing the Model

In [19]:
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.88      0.77        75
           1       0.72      0.43      0.53        54

    accuracy                           0.69       129
   macro avg       0.70      0.65      0.65       129
weighted avg       0.70      0.69      0.67       129

[[66  9]
 [31 23]]


In [ ]:
model.save('model/model_1.h5')